In [53]:
from db_handler import *
from api_handler import *
from config import *
from pandas import DataFrame
import json

In [54]:
connection_yelp = create_connection("./data/yelp_coctail_spots.sqlite")
connection_fsq = create_connection('./data/coctail_spots_old_port.sqlite')

In [55]:
select_coctail_spots_yelp = "SELECT * from yelp_coctail_spots"
coctail_spots_yelp = execute_read_query(connection_yelp, select_coctail_spots_yelp)

In [56]:
select_coctail_spots_fsq = 'SELECT * FROM coctail_spots_old_port'
coctail_spots_fsq = execute_read_query(connection_fsq, select_coctail_spots_fsq)

In [57]:
yelp_data = DataFrame(coctail_spots_yelp, 
                  columns=['PKID','id', 'name', 'rating', 'review_count', 'url', 'location'])

In [58]:
fsq_data = DataFrame(coctail_spots_fsq, columns=['PKID','fsq_id', 'name', 'popularity', 'rating',
                           'website', 'location', 'total_ratings'])

In [59]:
sorted_yelp = yelp_data.sort_values('rating', ascending=False)
sorted_fsq = fsq_data.sort_values('rating', ascending=False)

In [60]:
poi_yelp = sorted_yelp.head(10)
poi_fsq = sorted_fsq.head(10)

In [61]:
name_loc_yelp = poi_yelp[['name', 'location']]

In [62]:
name_loc_fsq = poi_fsq[['name', 'location']]

In [63]:
cartesian_fsq = name_loc_fsq.merge(name_loc_fsq, how='cross')
cartesian_yelp = name_loc_yelp.merge(name_loc_yelp, how='cross')

In [64]:
params_tuple_fsq = [({
        'origin': row['location_x'],
        'destination': row['location_y'],
        'mode': 'walking',
        'key': GOOGLE_API_KEY},
        (row['name_x'], row['name_y']))
            for index, row in cartesian_fsq.iterrows()]

params_tuple_yelp = [({
        'origin': row['location_x'],
        'destination': row['location_y'],
        'mode': 'walking',
        'key': GOOGLE_API_KEY},
        (row['name_x'], row['name_y']))
            for index, row in cartesian_yelp.iterrows()]

In [65]:
def travel_time(params):
    url = "https://maps.googleapis.com/maps/api/directions/json?"
    headers = {
        'Access': 'application/json',
    }
    travel_times = [(tup[1], 
                     get_req(url=url, params=tup[0], headers=headers).get('routes')[0].get('legs')[0].get('duration').get('value'))
                   for tup in params]
    return travel_times
    

In [66]:
travel_fsq = travel_time(params_tuple_fsq)

In [67]:
travel_yelp = travel_time(params_tuple_yelp)

In [68]:
travel_map_yelp = list(enumerate([travel_yelp[i][0][0] for i in range(0, 100, 10)]))

In [69]:
travel_map_fsq = list(enumerate([travel_fsq[i][0][0] for i in range(0, 100, 10)]))

In [70]:
distance_list_fsq = [city[1] for city in travel_fsq]
distance_list_yelp = [city[1] for city in travel_yelp]

In [111]:
def make_matrix(ordered_travel_distances):
    matrix = []
    for i in range(10):
        temp = [ordered_travel_distances[int(str(i) + str(j))] for j in range(10)]
        matrix.append(temp)
    return matrix

In [112]:
# fsq_matrix = make_matrix(distance_list_fsq)
# yelp_matrix = make_matrix(distance_list_yelp)

In [113]:
# fsq_matrix

In [114]:
# yelp_matrix

In [115]:
map_fsq = [travel_fsq[i][0][0] for i in range(0, 100, 10)]
map_yelp = [travel_yelp[i][0][0] for i in range(0, 100, 10)]

In [116]:
def create_data_model(ordered_travel_distances):
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = make_matrix(ordered_travel_distances)
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data

In [117]:
model_fsq = create_data_model(distance_list_fsq)  
model_yelp = create_data_model(distance_list_yelp)

In [118]:
model_fsq_json = json.dumps(model_fsq, indent=4)

In [79]:
model_yelp_json = json.dumps(model_yelp, indent=4)

### Saving Models to JSON

In [50]:
with open('./data/model_fsq.json', 'w') as f:
    f.write(model_fsq_json)

In [51]:
with open('././data/model_yelp.json', 'w') as f:
    f.write(model_fsq_json)

In [45]:
with open('./data/model_yelp.json', 'r') as f:
    model_yelp = json.load(f)

In [46]:
with open('./data/model_fsq.json', 'r') as f:
    model_fsq = json.load(f)

# Calculating Traveller Salesperson Problem

In [119]:
from itertools import permutations
from sys import maxsize

In [120]:
maxsize = maxsize

In [121]:
CITIES = 10

In [122]:
def travelling_salesman_person(matrix, source):
    path = []
    vertex = [i for i in range(CITIES)]

    min_path = maxsize
    next_permutation = permutations(vertex)
    for perm in next_permutation:
        # for v in vertex:
            current_path_weight = 0
            k = source
            cur_path = []
            for j in perm:
                current_path_weight += matrix[k][j]
                cur_path.append((k, j))
                k = j
            current_path_weight += matrix[k][source]
            if current_path_weight < min_path:
                min_path = current_path_weight
                path = cur_path

    return min_path, path

In [123]:
shortest_fsq = travelling_salesman_person(model_fsq['distance_matrix'], 3)

In [125]:
# shortest_fsq

In [86]:
shortest_yelp = travelling_salesman_person(model_yelp['distance_matrix'], 4)

In [108]:
# shortest_yelp

In [88]:
travel_path_fsq = [map_fsq[route[0]] for route in shortest_fsq[1]]

In [89]:
travel_path_yelp = [map_yelp[route[0]] for route in shortest_yelp[1]]

In [90]:
path_fsq = '  >>>  '.join(travel_path_fsq)

In [91]:
path_yelp = ' >>>  '.join(travel_path_yelp)

In [92]:
path_fsq

"Club Pelicano  >>>  Bar le Mal Nécessaire  >>>  Lab Quartier des Spectacles  >>>  Cafe Equitasse  >>>  El Pequeno Bar  >>>  Boho  >>>  L'Assommoir Notre-Dame  >>>  Bob's Cafe  >>>  Bord'Elle  >>>  Aruba"

In [93]:
path_yelp

'La Croisette >>>  Terrasse des Sucres >>>  Buvette Pastek >>>  Barroco >>>  Le Speakeasy >>>  Terrasse W >>>  Notkins >>>  The Coldroom >>>  El Pequeño Bar >>>  Pincette'